In [1]:
import matplotlib.pyplot as plt
%matplotlib inline 
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
#from sklearn import cross_validation, linear_model
from sklearn.model_selection import cross_validate
from sklearn.cluster import KMeans
from sklearn import datasets
import random
import time
import pylab as pl
from sklearn import metrics, ensemble
import xgboost as xgb
import seaborn as sns
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import scipy.sparse as sp
import tensorflow as tf
from tqdm import tqdm

In [2]:
df = pd.read_parquet("../user_track_df.parquet", engine='pyarrow')
df.head()

,user_id,song_id,listen_count,track_id,artist_name,track_name,track_uri,popularity,duration_ms,explicit,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBSUJE12A6D4F8CF5,2,TRPLAXZ128F4292406,Jorge Drexler,12 segundos de oscuridad,2ECKXkpPAxky87ohawpaeD,37,246826,0,...,-8.176,0,0.0327,0.11900,0.000412,0.103,0.0396,126.051,4,4ssUf5gLb1GBLxi1BhPrVt
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,TRHNCIR128F42334A5,Kanye West,Stronger,4fzsfWzRhPawzqhX8Qt9F3,82,311867,1,...,-7.858,0,0.1530,0.00564,0.000000,0.408,0.4900,103.992,4,5K4W6rqBFWDnAN6FQUkS6x
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,TRUATNS128F423457D,Kanye West,Stronger,4fzsfWzRhPawzqhX8Qt9F3,82,311867,1,...,-7.858,0,0.1530,0.00564,0.000000,0.408,0.4900,103.992,4,5K4W6rqBFWDnAN6FQUkS6x
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,TRYBNIB128F428E704,Jack Johnson,Constellations,3deZQXBY8CJFbrTc2PbU34,59,201640,0,...,-12.640,1,0.0355,0.46800,0.000043,0.117,0.4430,122.012,4,3GBPw9NK25X1Wt2OUvOwY3
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOEWFWM12A8C1308BA,1,TRLQPQJ128F42AA94F,Gipsy Kings,Soy,076jKe7yfP979o1QLKMIA2,47,189987,0,...,-12.321,0,0.0653,0.29700,0.000267,0.127,0.9050,114.656,4,3jc496ljiyrS3ECrD7QiqL


In [3]:
#we will implement BPR using matrix factorization, which means taking the interaction matrix R
#of size (all users x all times) and factoring it into matrix U of size (all users x latent features)
#and V of size (latent features x all items).  In the end we will have R ~ UxV

In [4]:
#Stochastic Gradient Descent (SGD) will be used in order to arrive at this approximation. 
#It works by initializing U and V with uniformly random values and then, using the optimization ciretrion
#from before, continuously updating them with new values tomaximize the posterior porbability
#i.e getting an increasing probability for R=UxVV

Feature engineering & Data Preprocessing

In [5]:
df = df.iloc[:20000]

In [6]:
#dropping unnecessary columns
df = df.drop(['track_id', 'track_uri'], axis=1)

#afterwards, we can see that the IDs given in the table are string types
#therefore, for this analysis we are going to drop them and instead encode the artist_name, track_name
#we are not going to drop user_id and we will encode it instead
df = df.drop(['song_id', 'artist_id'], axis=1)

In [7]:
#based on the boxplots presented in EDA we can observe that we might be dealing with outliers
#we will tackle this with IQR
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

listen_count            2.000000
popularity             22.000000
duration_ms         66240.000000
explicit                0.000000
danceability            0.217250
energy                  0.316000
key                     7.000000
loudness                4.317000
mode                    1.000000
speechiness             0.032500
acousticness            0.288082
instrumentalness        0.007700
liveness                0.165300
valence                 0.399000
tempo                  39.212000
time_signature          0.000000
dtype: float64


In [8]:
df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
df.shape

(9777, 20)

In [9]:
#feature importance

In [10]:
# Convert artists names into numerical IDs
df['user_id_num'] = df['user_id'].astype("category").cat.codes
df['artist_id'] = df['artist_name'].astype("category").cat.codes
df['track_id'] = df['track_name'].astype("category").cat.codes

In [11]:
#drop columns that have categorical features such as the name of the artist and track.
#usually we would encode this; however, there's no need as we already have the ids and this way
#we would just have several columns depicting same values that is unnecessary

#before we do this we will create a lookup frame so we can get the artist names back in readable form later
item_lookup = df[['artist_id', 'artist_name']].drop_duplicates()
item_lookup['artist_id'] = item_lookup.artist_id.astype(str)

#item_lookup_track = df[['track_id', 'track_name']].drop_duplicates()
#item_lookup_track['track_id'] = item_lookup_track.track_id.astype(str)

#dropping the columns
df = df.drop(['track_name', 'artist_name', 'user_id'], axis=1)

In [12]:
item_lookup = item_lookup.rename(columns={'A': 'X'})

In [13]:
df = df.rename(columns={'artist_name': 'artist'})

In [14]:
#checking the columns
df.columns

Index(['listen_count', 'popularity', 'duration_ms', 'explicit', 'release_date',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'user_id_num', 'artist_id', 'track_id'],
      dtype='object')

In [15]:
# Drop any rows with 0 plays
df = df.loc[df.listen_count != 0]

In [16]:
# Create lists of all users, artists and plays
users = list(np.sort(df.user_id_num.unique()))
artists = list(np.sort(df.artist_id.unique()))
plays = list(df.listen_count)

In [17]:
# Get the rows and columns for our new matrix
rows = df.user_id_num.astype(float)
cols = df.artist_id.astype(float)

In [18]:
# Contruct a sparse matrix for our users and items containing number of plays
data_sparse = sp.csr_matrix((plays, (rows, cols)), shape=(len(users), len(artists)))

In [19]:
# Get the values of our matrix as a list of user ids
# and item ids. Note that our lists have the same length
# as each user id repeats one time for each played artist.
uids, iids = data_sparse.nonzero()

Defining Hyperparameters

In [34]:
#-------------
# HYPERPARAMS
#-------------

epochs = 50 #50
batches = 30 #30
num_factors = 64 # Number of latent features

# Independent lambda regularization values 
# for user, items and bias.
lambda_user = 0.0000001
lambda_item = 0.0000001
lambda_bias = 0.0000001

# Our learning rate 
lr = 0.005

# How many (u,i,j) triplets we sample for each batch
samples = 1000

Defining Tensorflow Graph

In [35]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#-------------------------
# TENSORFLOW GRAPH
#-------------------------

# Set up our Tensorflow graph
graph = tf.Graph()

def init_variable(size, dim, name=None):
    '''
    Helper function to initialize a new variable with
    uniform random values.
    '''
    std = np.sqrt(2 / dim)
    return tf.Variable(tf.random.uniform([size, dim], -std, std), name=name)


def embed(inputs, size, dim, name=None):
    '''
    Helper function to get a Tensorflow variable and create
    an embedding lookup to map our user and item
    indices to vectors.
    '''
    emb = init_variable(size, dim, name)
    return tf.nn.embedding_lookup(emb, inputs)


def get_variable(graph, session, name):
    '''
    Helper function to get the value of a
    Tensorflow variable by name.
    '''
    v = graph.get_operation_by_name(name)
    v = v.values()[0]
    v = v.eval(session=session)
    return v

with graph.as_default():
    '''
    Loss function: 
    -SUM ln σ(xui - xuj) + λ(w1)**2 + λ(w2)**2 + λ(w3)**2 ...
    ln = the natural log
    σ(xuij) = the sigmoid function of xuij.
    λ = lambda regularization value.
    ||W||**2 = the squared L2 norm of our model parameters.
    '''

    # Input into our model, in this case our user (u),
    # known item (i) an unknown item (i) triplets.
    u = tf.placeholder(tf.int32, shape=(None, 1))
    i = tf.placeholder(tf.int32, shape=(None, 1))
    j = tf.placeholder(tf.int32, shape=(None, 1))

    # User feature embedding
    u_factors = tf.Variable(tf.random_normal([len(users), num_factors]), name='user_factors') # U matrix

    # Known and unknown item embeddings
    item_factors = tf.Variable(tf.random_normal([len(artists), num_factors]), name='item_factors') # V matrix
    i_factors = tf.nn.embedding_lookup(item_factors, i)
    j_factors = tf.nn.embedding_lookup(item_factors, j)

    # i and j bias embeddings.
    item_bias = tf.Variable(tf.zeros([len(artists), 1]), name='item_bias')
    i_bias = tf.nn.embedding_lookup(item_bias, i)
    i_bias = tf.reshape(i_bias, [-1, 1])
    j_bias = tf.nn.embedding_lookup(item_bias, j)
    j_bias = tf.reshape(j_bias, [-1, 1])

    # Calculate the dot product + bias for known and unknown
    # item to get xui and xuj.
    xui = i_bias + tf.reduce_sum(tf.multiply(u_factors, i_factors), axis=2)
    xuj = j_bias + tf.reduce_sum(tf.multiply(u_factors, j_factors), axis=2)

    # We calculate xuij.
    xuij = xui - xuj

    # Calculate the mean AUC (area under curve).
    # if xuij is greater than 0, that means that 
    # xui is greater than xuj (and thats what we want).
    u_auc = tf.reduce_mean(tf.cast(tf.greater(xuij, 0), tf.float32))

    # Output the AUC value to tensorboard for monitoring.
    tf.summary.scalar('auc', u_auc)

    # Calculate the squared L2 norm ||W||**2 multiplied by λ.
    l2_norm = tf.add_n([
        lambda_user * tf.reduce_sum(tf.multiply(u_factors, u_factors)),
        lambda_item * tf.reduce_sum(tf.multiply(i_factors, i_factors)),
        lambda_item * tf.reduce_sum(tf.multiply(j_factors, j_factors)),
        lambda_bias * tf.reduce_sum(tf.multiply(i_bias, i_bias)),
        lambda_bias * tf.reduce_sum(tf.multiply(j_bias, j_bias))
        ])

    # Calculate the loss as -ln σ(Xuij) + ||W||**2
    loss = -tf.reduce_mean(tf.log(tf.sigmoid(xuij))) + l2_norm

    # Train using the Adam optimizer to minimize 
    # our loss function.
    opt = tf.train.AdamOptimizer(learning_rate=lr)
    step = opt.minimize(loss)

    # Initialize all tensorflow variables.
    init = tf.global_variables_initializer()

    #-----------------------
    # FIND SIMILAR ARTISTS
    #-----------------------

    def find_similar_artists(artist=None, num_items=10):
        """Find artists similar to an artist.
        Args:
            artist (str): The name of the artist we want to find similar artists for
            num_items (int): How many similar artists we want to return.
        Returns:
            similar (pandas.DataFrame): DataFrame with num_items artist names and scores
        """

        # Grab our User matrix U
        user_vecs = get_variable(graph, sess, 'user_factors')

        # Grab our Item matrix V
        item_vecs = get_variable(graph, sess, 'item_factors')

        # Grab our item bias
        item_bi = get_variable(graph, sess, 'item_bias').reshape(-1)

        # Get the item id for Lady GaGa
        item_id = int(item_lookup[item_lookup.artist_name == artist]['artist_id'])

        # Get the item vector for our item_id and transpose it.
        item_vec = item_vecs[item_id].T

        # Calculate the similarity between Lady GaGa and all other artists
        # by multiplying the item vector with our item_matrix
        scores = np.add(item_vecs.dot(item_vec), item_bi).reshape(1,-1)[0]

        # Get the indices for the top 10 scores
        top_10 = np.argsort(scores)[::-1][:num_items]

        # We then use our lookup table to grab the names of these indices
        # and add it along with its score to a pandas dataframe.
        artists, artist_scores = [], []
        
        for idx in top_10:
            artists.append(item_lookup.artist_name.loc[item_lookup.artist_id == str(idx)].iloc[0])
            artist_scores.append(scores[idx])

        similar = pd.DataFrame({'artist': artists, 'score': artist_scores})

        return similar

    #---------------------
    # MAKE RECOMMENDATION
    #---------------------

    def make_recommendation(user_id=None, num_items=10):
        """Recommend items for a given user given a trained model
        Args:
            user_id (int): The id of the user we want to create recommendations for.
            num_items (int): How many recommendations we want to return.
        Returns:
            recommendations (pandas.DataFrame): DataFrame with num_items artist names and scores
        """

        # Grab our user matrix U
        user_vecs = get_variable(graph, sess, 'user_factors')

        # Grab our item matrix V
        item_vecs = get_variable(graph, sess, 'item_factors')

        # Grab our item bias
        item_bi = get_variable(graph, sess, 'item_bias').reshape(-1)

        # Calculate the score for our user for all items. 
        rec_vector = np.add(user_vecs[user_id, :].dot(item_vecs.T), item_bi)

        # Grab the indices of the top users
        item_idx = np.argsort(rec_vector)[::-1][:num_items]

        # Map the indices to artist names and add to dataframe along with scores.
        artists, scores = [], []

        for idx in item_idx:
            artists.append(item_lookup.artist_name.loc[item_lookup.artist_id == str(idx)].iloc[0])
            scores.append(rec_vector[idx])

        recommendations = pd.DataFrame({'artist': artists, 'score': scores})

        return recommendations


    #------------------
    # GRAPH EXECUTION
    #------------------

    # Run the session with the constructed graph.
    with tf.Session(config=None, graph=graph) as sess:
        # Initialize the variables in the graph.
        sess.run(init)

        # This has nothing to do with TensorFlow but gives
        # us a nice progress bar for the training.
        progress = tqdm(total=batches*epochs)

        for _ in range(epochs):
            for _ in range(batches):
                # We want to sample one known and one unknown 
                # item for each user. 

                # First we sample 15000 uniform indices.
                idx = np.random.randint(low=0, high=len(uids), size=samples)

                # We then grab the users matching those indices.
                batch_u = uids[idx].reshape(-1, 1)

                # Then the known items for those users.
                batch_i = iids[idx].reshape(-1, 1)

                # Lastly we randomly sample one unknown item for each user.
                batch_j = np.random.randint(
                        low=0, high=len(artists), size=(samples, 1), dtype='int32')

                # Feed our users, known and unknown items to
                # our tensorflow graph. 
                feed_dict = { u: batch_u, i: batch_i, j: batch_j }

                # We run the session.
                _, l, auc = sess.run([step, loss, u_auc], feed_dict)

            progress.update(batches)
            progress.set_description('Loss: %.3f | AUC: %.3f' % (l, auc))

        progress.close()

        print("\n")

        print("-------Similar Artists-------")

        print(find_similar_artists(artist='Coldplay'))

        print("\n")

        print("-------Recommendations-------")

        print(make_recommendation(user_id=0))

Loss: 0.377 | AUC: 0.844: 100%|██████████| 1500/1500 [00:56<00:00, 26.49it/s]



-------Similar Artists-------
            artist      score
0         Coldplay  56.910358
1      Evanescence  27.507435
2      The Killers  25.177402
3        Radiohead  25.068197
4          Cartola  25.054844
5   The Black Keys  24.249895
6      Miley Cyrus  23.838608
7          Rihanna  23.540762
8  Jimmy Eat World  22.593998
9         Bon Jovi  22.531239


-------Recommendations-------
                   artist     score
0                Coldplay  3.709384
1             OneRepublic  3.541508
2                 Cartola  3.215309
3                Bon Jovi  3.115548
4            Taylor Swift  3.090174
5           Justin Bieber  3.059061
6                 La Roux  2.971563
7             The Killers  2.906898
8             Miley Cyrus  2.905311
9  Florence + The Machine  2.823690
